In [25]:

import pandas as pd
import numpy as np
import math as m
import os
import csv
import random as rn
from random import randint
from scipy.stats import beta

# Quantities to seed

papers_number = 10000
readers_number = 5000
authors_number = 25

papers = np.arange(papers_number)
readers = np.arange(readers_number)
authors = np.arange(authors_number)

# Seed folder path

dataset_name = "seed_2/r_4_beta"
dataset_folder_path = f"../data/{dataset_name}/"
info_file_path = f"{dataset_folder_path}info.csv"
ratings_file_path = f"{dataset_folder_path}ratings.csv"
authors_file_path = f"{dataset_folder_path}authors.csv"

os.makedirs(dataset_folder_path, exist_ok=True)

print("DATASET NAME: ", dataset_name)
print("DATASET FOLDER PATH: ", dataset_folder_path)
print("INFO FILE PATH: ", info_file_path)
print("RATINGS FILE PATH: ", ratings_file_path)
print("AUTHORS FILE PATH: ", authors_file_path)

# Papers distribution generation with beta distribution

print("---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------")

# CASE 1: a == b == 1, 5% of papers
beta_distributions_frequencies = [(m.floor((5*papers_number)/100), (1, 1))]
# CASE 2: a == b > 1, 30% of papers
a = randint(2, 10)
b = a
beta_distributions_frequencies.append((m.floor((30*papers_number)/100), (a, b)))
# CASE 3: 0 < (a ^ b) < 1, 30% of papers
a = rn.uniform(0.001, 1)
b = rn.uniform(0.001, 1)
beta_distributions_frequencies.append((m.floor((20*papers_number)/100), (a, b)))
# CASE 4: (a V b) == 1, (a > b V b > a), 20% of papers
a = 1
b = randint(1, 10)
if rn.randint(0,1) > 0.5:
    a, b = b, a
beta_distributions_frequencies.append((m.floor((30*papers_number)/100), (a, b)))
# CASE 5: (a ^ b) > 1, (a > b V b > a), 15% of papers
a = randint(2, 10)
b = randint(2 + a, 10 + a)
if rn.randint(0,1) > 0.5:
    a, b = b, a
beta_distributions_frequencies.append((m.floor((15*papers_number)/100), (a, b)))

papers_set = set(papers)
paper_distributions = [None] * papers_number

generated_papers_distributions = 0
for (papers_amount, (a, b)) in beta_distributions_frequencies:
    current_paper_set = rn.sample(papers_set, papers_amount)
    for paper in current_paper_set:
        percentage = 100*generated_papers_distributions/papers_number
        if percentage % 10 == 0:
            print(f"{int(generated_papers_distributions)}/{papers_number} ({int(percentage)}/100%)")
        paper_distributions[paper] = beta(a=a, b=b)
        generated_papers_distributions = generated_papers_distributions + 1
        papers_set.remove(paper)
print(f"{papers_number}/{papers_number} (100/100%)")

print("---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------")


DATASET NAME:  seed_2/r_4_beta
DATASET FOLDER PATH:  ../data/seed_2/r_4_beta/
INFO FILE PATH:  ../data/seed_2/r_4_beta/info.csv
RATINGS FILE PATH:  ../data/seed_2/r_4_beta/ratings.csv
AUTHORS FILE PATH:  ../data/seed_2/r_4_beta/authors.csv
---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------
0/10000 (0/100%)
1000/10000 (10/100%)
2000/10000 (20/100%)
3000/10000 (30/100%)
4000/10000 (40/100%)
5000/10000 (50/100%)
6000/10000 (60/100%)
7000/10000 (70/100%)
8000/10000 (80/100%)
9000/10000 (90/100%)
10000/10000 (100/100%)
---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------


In [26]:

# Ratings file generation

# N sets of readers, each one has X% of the total

readers_percent = 20
reader_sets_number = m.floor(100 / readers_percent)
readers_amount = m.floor((readers_number*readers_percent)/100)

readers_set = set(readers)
readers_sets = []

# Readers rate papers with a certain frequence

paper_frequencies = [2, 4, 8, 30, 90]

print("---------- READERS SETS GENERATION STARTED ----------")

ratings_number = sum(paper_frequencies) * readers_amount
for x in range(0, reader_sets_number):
    current_readers_set = rn.sample(readers_set, readers_amount)
    # Removing last index
    if readers_number in current_readers_set: current_readers_set.remove(readers_number)
    readers_sets.append(current_readers_set)
    for reader in current_readers_set:
        readers_set.remove(reader)
    print(f"SET {x}: ", current_readers_set)
     
print("---------- READERS SETS GENERATION COMPLETED ----------")

print("---------- RATINGS GENERATION STARTED ----------")

generated_ratings = 0
rated_papers = []
rated_readers = []
with open(ratings_file_path, mode='w', newline='') as ratings_file:
    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ratings_writer.writerow(['Timestamp', 'Reader', 'Paper', 'Score'])
    for current_set in range(0, reader_sets_number):
        paper_per_reader = paper_frequencies[current_set]
        readers_set = readers_sets[current_set]
        for reader in readers_set:
            for index, paper in enumerate(rn.sample(set(papers), paper_per_reader)):
                paper_distribution = paper_distributions[paper]
                percentage = 100*generated_ratings/ratings_number
                if percentage % 10 == 0:
                    print(f"{int(generated_ratings)}/{ratings_number} ({int(percentage)}/100%)")
                current_tuple = {
                    "Reader": reader, 
                    "Paper": paper, 
                    "Score": round(paper_distribution.rvs(1)[0], 2), 
                }
                ratings_writer.writerow([generated_ratings, current_tuple["Reader"], current_tuple["Paper"], current_tuple["Score"]])
                generated_ratings+=1
                rated_readers.append(reader)
                rated_papers.append(paper)
    
    # Filling gaps
    unrated_papers = set(papers) - set(rated_papers)    
    for paper in unrated_papers:
        for reader in rn.sample(set(readers), 3): 
            paper_distribution = paper_distributions[paper]
            current_tuple = {
                "Reader": reader, 
                "Paper": paper, 
                "Score": round(paper_distribution.rvs(1)[0], 2), 
            }
            ratings_writer.writerow([generated_ratings, current_tuple["Reader"], current_tuple["Paper"], current_tuple["Score"]])    
            generated_ratings = generated_ratings + 1

    print(f"{ratings_number}/{ratings_number} (100/100%)")
    
ratings_file.close()

print("---------- RATINGS GENERATION ENDED ----------")


---------- READERS SETS GENERATION STARTED ----------
SET 0:  [1145, 3447, 4311, 2094, 1514, 2993, 4142, 2595, 18, 1278, 3096, 4725, 3335, 4393, 2366, 912, 4943, 3039, 4059, 1400, 3559, 569, 1257, 4983, 906, 3715, 2087, 1183, 4514, 1829, 2888, 686, 2288, 3998, 485, 1885, 4458, 3274, 4964, 1124, 2931, 3438, 2842, 3607, 4482, 1546, 1701, 1368, 2530, 450, 3184, 2542, 1473, 522, 1306, 3895, 1211, 4374, 4234, 3777, 4061, 426, 4057, 1613, 2404, 4210, 4987, 4950, 2101, 3759, 2355, 1960, 813, 3803, 962, 1605, 771, 701, 712, 3545, 4280, 551, 3795, 2546, 3865, 2229, 4485, 3508, 861, 4286, 3407, 463, 4869, 633, 3296, 4459, 2463, 2644, 4153, 2162, 888, 3538, 3760, 349, 2122, 4677, 3470, 2672, 3926, 2780, 3656, 308, 3388, 2467, 4098, 1972, 208, 4605, 772, 1161, 2829, 3024, 249, 3972, 3189, 4392, 3066, 215, 1340, 3821, 4336, 2953, 2855, 1136, 4708, 3929, 2248, 3794, 1287, 324, 4318, 2943, 4698, 2164, 111, 577, 432, 1410, 213, 3553, 1855, 3399, 3315, 679, 1460, 3838, 3408, 3666, 4667, 1134, 3975, 335

In [27]:

# Authors file generation

print("---------- AUTHORS GENERATION STARTED ----------")

with open(authors_file_path, mode='w', newline='') as authors_file:
    authors_writer = csv.writer(authors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    authors_writer.writerow(["Author", "Paper"])
    for index, author in enumerate(authors):
        percentage = 100*index/authors_number
        if percentage % 10 == 0:
            print(f"{int(index)}/{authors_number} ({int(percentage)}/100%)")
        # An author writes a number of paper between 1 and paper_fraction
        author_papers_number = rn.randint(1, (papers_number-1))
        papers_written = np.random.choice(papers, author_papers_number).tolist()
        papers_written = set(papers_written)
        if len(papers_written) > 1:
            papers_written = map(str, list(papers_written))
            papers_written = ";".join(papers_written)
        authors_writer.writerow([author, papers_written])
    print(f"{authors_number}/{authors_number} (100/100%)")
authors_file.close()
        
print("---------- AUTHORS GENERATION ENDED ----------")


---------- AUTHORS GENERATION STARTED ----------
0/25 (0/100%)
5/25 (20/100%)
10/25 (40/100%)
15/25 (60/100%)
20/25 (80/100%)
25/25 (100/100%)
---------- AUTHORS GENERATION ENDED ----------


In [28]:

# Info file generation

print("---------- INFO GENERATION STARTED ----------")

info_dataframe = pd.DataFrame(columns=["Dataset", "Paper", "Reader", "Rating", "Author"])
info_dataframe = info_dataframe.append(
    {
        "Dataset": dataset_name, 
        "Paper": papers_number, 
        "Reader": readers_number, 
        "Rating": ratings_number, 
        "Author": authors_number
    }, ignore_index=True)
info_dataframe.to_csv(info_file_path, index=False)

print("---------- INFO GENERATION ENDED ----------")

---------- INFO GENERATION STARTED ----------
---------- INFO GENERATION ENDED ----------
